<a href="https://colab.research.google.com/github/S-ArunaDevi06/GastrointestinalCancerAI/blob/main/GIAI_BERT_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BERT**

For direct questions


In [ ]:
!pip install transformers torch
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import torch
import numpy as np

In [ ]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import numpy as np


tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def split_passage(passage, split_points):
    parts = []
    last_index = 0

    for split_point in split_points:
        split_index = passage.find(split_point)
        if split_index == -1:
            continue

        if last_index != split_index:
            parts.append(passage[last_index:split_index].strip())
        last_index = split_index

    if last_index < len(passage):
        parts.append(passage[last_index:].strip())

    return parts

#for processing each part of the chunk
def process_chunk(question, chunk, max_len):
    question_tokens = tokenizer_for_bert.encode(question, add_special_tokens=False)
    chunk_tokens = tokenizer_for_bert.encode(chunk, add_special_tokens=False)

    chunk_tokens = chunk_tokens[:max_len - len(question_tokens) - 3]

    input_ids = [tokenizer_for_bert.cls_token_id] + question_tokens + [tokenizer_for_bert.sep_token_id] + chunk_tokens + [tokenizer_for_bert.sep_token_id]
    attention_mask = [1] * len(input_ids)

    if len(input_ids) > max_len:
        input_ids = input_ids[:max_len]
        attention_mask = attention_mask[:max_len]

    inputs = {
        'input_ids': torch.tensor([input_ids]),
        'attention_mask': torch.tensor([attention_mask])
    }

    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_scores = start_scores.detach().numpy().flatten()
    end_scores = end_scores.detach().numpy().flatten()

    answer_start_index = np.argmax(start_scores)
    answer_end_index = np.argmax(end_scores)

    tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids)

    answer = tokens[answer_start_index]
    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i].startswith('##'):
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]



    print(f"Tokens: {tokens}")
    print(f"Answer Start Index: {answer_start_index}")
    print(f"Answer End Index: {answer_end_index}")
    print(f"Answer: {answer}")

    if (start_scores[answer_start_index] < 0) or (answer_start_index == 0) or (answer_end_index < answer_start_index) or (answer == '[SEP]'):
        answer = "Sorry!, I was unable to discover an answer in the passage."

    return answer, start_scores[answer_start_index] + end_scores[answer_end_index]

def bert_question_answer(question, passage, split_points, max_len=512):
    parts = split_passage(passage, split_points)

    answers = []
    scores = []

    for part in parts:
        answer, score = process_chunk(question, part, max_len)
        answers.append(answer)
        scores.append(score)

    # Determine which part has the best score
    if scores:
        best_answer = answers[np.argmax(scores)]
    else:
        best_answer = "Sorry!, I was unable to discover an answer in the passage."

    return best_answer

#example
passage = "Description: Esophageal cancer is cancer arising from the esophagus—the food pipe that runs between the throat and the stomach. Causes: Esophageal cancer may arise due to intake of tobacco, alcohol, very hot drinks, poor diet, and chewing betel nut. The most common causes of the adenocarcinoma type are smoking tobacco, obesity, and acid reflux. Symptoms: The first and the most common symptom is usually difficulty in swallowing, Pain when swallowing , Weight loss ,Pain behind the breastbone or in the region around the stomach often feels like heartburn,  an unusually husky, raspy, or hoarse-sounding cough, a result of the tumor affecting the recurrent laryngeal nerve, vomiting of blood and Hypercalcemia (excess calcium in the blood). Diagnosis: The disease is diagnosed by biopsy done by an endoscope. Clinical evaluation: diagnosis is best made with an examination using an endoscope. Computed tomography (CT) of the chest, abdomen and pelvis can evaluate whether the cancer has spread to adjacent tissues or distant organs. Positron emission tomography is also used to estimate the extent of the disease and is regarded as more precise than CT alone. Staging: Staging is based on the TNM staging system, which classifies the amount of tumor invasion (T), involvement of lymph nodes (N), and distant metastasis (M).Genetic mutations: TP53 (p53): TP53 is a tumor suppressor gene that plays a crucial role in preventing abnormal cell growth. CDKN2A (p16INK4a): CDKN2A encodes the p16INK4a protein, which regulates the cell cycle by inhibiting cyclin-dependent kinases. NOTCH1:NOTCH1 is involved in cell differentiation and proliferation. Mutations in NOTCH1 have been found in ESCC(esophageal squamous cell carcinoma). SMAD4 (DPC4): SMAD4 is part of the TGF-β signaling pathway, which controls cell growth and differentiation. Mutations in SMAD4 are linked to esophageal adenocarcinoma. PIK3CA:PIK3CA encodes a protein involved in cell growth and survival. Mutations in PIK3CA are seen in both ESCC and adenocarcinoma.HER2:Some esophagus cancers have too much of the HER2 protein on the surface of their cells, which can help cancer cells grow.  Treatments are: Surgery: If the cancer has been diagnosed while still in an early stage, surgical treatment with a curative intention may be possible. curative surgery of early-stage lesions may entail removal of all or part of the esophagus (esophagectomy). Chemotherapy and radiotherapy: Chemotherapy depends on the tumor type, but tends to be cisplatin-based (or carboplatin or oxaliplatin) every three weeks with fluorouracil (5-FU) either continuously or every three weeks. Radiotherapy is given before, during, or after chemotherapy or surgery, and sometimes on its own to control symptoms. In patients with localized disease but contraindications to surgery, radical radiotherapy may be used with curative intent. Laser therapy is the use of high-intensity light to destroy tumor cells while affecting only the treated area. This is typically done if the cancer cannot be removed by surgery. Photodynamic therapy, a type of laser therapy, involves the use of drugs that are absorbed by cancer cells; when exposed to a special light, the drugs become active and destroy the cancer cells."

question = "What are the diagnosis for esophageal cancer"
split_points = ["Symptoms:", "Genetic mutations:"]
print(passage.find("Genetic mutations:"))
print(bert_question_answer(question, passage, split_points))


Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1325
Tokens: ['[CLS]', 'what', 'are', 'the', 'diagnosis', 'for', 'es', '##op', '##ha', '##ge', '##al', 'cancer', '[SEP]', 'description', ':', 'es', '##op', '##ha', '##ge', '##al', 'cancer', 'is', 'cancer', 'arising', 'from', 'the', 'es', '##op', '##ha', '##gus', '—', 'the', 'food', 'pipe', 'that', 'runs', 'between', 'the', 'throat', 'and', 'the', 'stomach', '.', 'causes', ':', 'es', '##op', '##ha', '##ge', '##al', 'cancer', 'may', 'arise', 'due', 'to', 'intake', 'of', 'tobacco', ',', 'alcohol', ',', 'very', 'hot', 'drinks', ',', 'poor', 'diet', ',', 'and', 'chewing', 'bet', '##el', 'nut', '.', 'the', 'most', 'common', 'causes', 'of', 'the', 'aden', '##oca', '##rc', '##ino', '##ma', 'type', 'are', 'smoking', 'tobacco', ',', 'obesity', ',', 'and', 'acid', 'ref', '##lux', '.', '[SEP]']
Answer Start Index: 22
Answer End Index: 41
Answer: cancer arising from the esophagus — the food pipe that runs between the throat and the stomach
Tokens: ['[CLS]', 'what', 'are', 'the', 'diagnosis', 'for',

In [ ]:
ans=bert_question_answer("What are the treatments for esophageal cancer", "Description: Esophageal cancer is cancer arising from the esophagus—the food pipe that runs between the throat and the stomach. Causes: Esophageal cancer may arise due to intake of tobacco, alcohol, very hot drinks, poor diet, and chewing betel nut. The most common causes of the adenocarcinoma type are smoking tobacco, obesity, and acid reflux. Symptoms: The first and the most common symptom is usually difficulty in swallowing, Pain when swallowing , Weight loss ,Pain behind the breastbone or in the region around the stomach often feels like heartburn,  an unusually husky, raspy, or hoarse-sounding cough, a result of the tumor affecting the recurrent laryngeal nerve, vomiting of blood and Hypercalcemia (excess calcium in the blood). Diagnosis: The disease is diagnosed by biopsy done by an endoscope. Clinical evaluation: diagnosis is best made with an examination using an endoscope. Computed tomography (CT) of the chest, abdomen and pelvis can evaluate whether the cancer has spread to adjacent tissues or distant organs. Positron emission tomography is also used to estimate the extent of the disease and is regarded as more precise than CT alone. Staging: Staging is based on the TNM staging system, which classifies the amount of tumor invasion (T), involvement of lymph nodes (N), and distant metastasis (M).Genetic mutations: TP53 (p53): TP53 is a tumor suppressor gene that plays a crucial role in preventing abnormal cell growth. CDKN2A (p16INK4a): CDKN2A encodes the p16INK4a protein, which regulates the cell cycle by inhibiting cyclin-dependent kinases. NOTCH1:NOTCH1 is involved in cell differentiation and proliferation. Mutations in NOTCH1 have been found in ESCC(esophageal squamous cell carcinoma). SMAD4 (DPC4): SMAD4 is part of the TGF-β signaling pathway, which controls cell growth and differentiation. Mutations in SMAD4 are linked to esophageal adenocarcinoma. PIK3CA:PIK3CA encodes a protein involved in cell growth and survival. Mutations in PIK3CA are seen in both ESCC and adenocarcinoma.HER2:Some esophagus cancers have too much of the HER2 protein on the surface of their cells, which can help cancer cells grow.  Treatments are: Surgery: If the cancer has been diagnosed while still in an early stage, surgical treatment with a curative intention may be possible. curative surgery of early-stage lesions may entail removal of all or part of the esophagus (esophagectomy). Chemotherapy and radiotherapy: Chemotherapy depends on the tumor type, but tends to be cisplatin-based (or carboplatin or oxaliplatin) every three weeks with fluorouracil (5-FU) either continuously or every three weeks. Radiotherapy is given before, during, or after chemotherapy or surgery, and sometimes on its own to control symptoms. In patients with localized disease but contraindications to surgery, radical radiotherapy may be used with curative intent. Laser therapy is the use of high-intensity light to destroy tumor cells while affecting only the treated area. This is typically done if the cancer cannot be removed by surgery. Photodynamic therapy, a type of laser therapy, involves the use of drugs that are absorbed by cancer cells; when exposed to a special light, the drugs become active and destroy the cancer cells.",split_points)
print("ans: ",ans)

ans:  chemotherapy and radiotherapy


In [ ]:
ans=bert_question_answer("What are treatments for pancreatic cancer", "Description: Pancreatic cancer arises when cells in the pancreas.A number of types of pancreatic cancer are known. Several other types of cancer, which collectively represent the majority of the non-adenocarcinomas, can also arise from these cells.Causes : Risk factors for pancreatic cancer include tobacco smoking, obesity, diabetes, and certain rare genetic conditions. Symptoms:Pain in the upper abdomen or back, often spreading from around the stomach to the back, Jaundice, weight loss, Trousseau's syndrome – in which blood clots form spontaneously in the portal blood vessels , weakness and tiring easily, dry mouth, sleep problems, and a palpable abdominal mass. Diagnosis: Histopathology-The most common form of pancreatic cancer (adenocarcinoma) is typically characterized by moderately to poorly differentiated glandular structures on microscopic examination. Staging: Exocrine cancers: Pancreatic cancer is usually staged following a CT scan. Genetic mutations include: Pancreatic ductal adenocarcinoma (PDAC):p53, SMAD4, KRAS, p16. Pancreatic acinar cell carcinoma (ACC): p53, SMAD4, ARID1A, GNAS. Solid pseudopapillary tumor:Point mutation in exon 3 of β-catenin gene. Adenosquamous carcinoma: KRAS, p53. Pancreatic neuroendocrine tumor: MEN1, DAXX/ATRX. p53:a tumor suppressor protein encoded by the TP53 gene that responds to DNA damage by regulating cell-cycle arrest, apoptosis, and senescence. SMAD4: It is also known as deleted in pancreatic carcinoma 4 or DPC4, is located on chromosome 18q21 and inactivated in roughly 55% of pancreatic cancers. ARID1A: It a subunit of the SWI/SNF complex, which serves as a tumor suppressor gene commonly absent in subsets of pancreatic cancer and IPMN patients, and low expression of Arid1a is associated with poor overall survival of pancreatic cancer. GNAS: The day-to-day function of the GNAS gene is to maintain body glucose levels and assist in normal development, but when mutated, it becomes overactive and causes cancer. KRAS: Kirsten RAt Sarcoma virus,KRAS is involved in the regulation of cell division as a result of its ability to relay external signals to the cell nucleus MEN1: Multiple endocrine neoplasia type 1 (MEN1) is a rare endocrine tumor syndrome with high penetrance. BRCA: The mutations BRCA1 and BRCA2 increase a person’s lifetime risk of developing breast.Their normal function is to repair damage to DNA, but when BRCA1 or BRCA2 is mutated and doesn’t work correctly, the accumulation of unrepaired DNA damage can ultimately lead to unregulated cell growth, or cancer.  Treatments: Surgery: For cancers involving the head of the pancreas, the Whipple procedure is the most commonly attempted curative surgical treatment. if the cancer is invading or compressing the duodenum or colon, bypass surgery might overcome the obstruction and improve quality of life. Chemotherapy: After surgery, adjuvant chemotherapy with gemcitabine or 5-FU can be offered if the person is sufficiently fit, after a recovery period of one to two months.Radiotherapy: Radiotherapy may form part of treatment to attempt to shrink a tumor to a resectable state, but its use on unresectable tumors remains controversial as there are conflicting results from clinical trials. ",split_points)
print("ans: ",ans)

Tokens: ['[CLS]', 'what', 'are', 'treatments', 'for', 'pan', '##cre', '##atic', 'cancer', '[SEP]', 'description', ':', 'pan', '##cre', '##atic', 'cancer', 'arises', 'when', 'cells', 'in', 'the', 'pan', '##cre', '##as', '.', 'a', 'number', 'of', 'types', 'of', 'pan', '##cre', '##atic', 'cancer', 'are', 'known', '.', 'several', 'other', 'types', 'of', 'cancer', ',', 'which', 'collectively', 'represent', 'the', 'majority', 'of', 'the', 'non', '-', 'aden', '##oca', '##rc', '##ino', '##mas', ',', 'can', 'also', 'arise', 'from', 'these', 'cells', '.', 'causes', ':', 'risk', 'factors', 'for', 'pan', '##cre', '##atic', 'cancer', 'include', 'tobacco', 'smoking', ',', 'obesity', ',', 'diabetes', ',', 'and', 'certain', 'rare', 'genetic', 'conditions', '.', '[SEP]']
Answer Start Index: 65
Answer End Index: 86
Answer: causes : risk factors for pancreatic cancer include tobacco smoking , obesity , diabetes , and certain rare genetic conditions
Tokens: ['[CLS]', 'what', 'are', 'treatments', 'for', 'p

**Questions from Doctor**

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import numpy as np


tokenizer_for_bert = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

def split_passage(passage, split_points):
    parts = []
    last_index = 0

    for split_point in split_points:
        split_index = passage.find(split_point)
        if split_index == -1:
            continue

        if last_index != split_index:
            parts.append(passage[last_index:split_index].strip())
        last_index = split_index

    if last_index < len(passage):
        parts.append(passage[last_index:].strip())

    return parts

def simplify_question(question):
    # Simplify the question by removing unnecessary parts
    simplified_question = question
    if "Can you provide information on" in question:
        simplified_question = question.split("Can you provide information on")[1].strip()
    return simplified_question

def process_chunk(question, chunk, max_len):
    question_tokens = tokenizer_for_bert.encode(question, add_special_tokens=False)
    chunk_tokens = tokenizer_for_bert.encode(chunk, add_special_tokens=False)

    chunk_tokens = chunk_tokens[:max_len - len(question_tokens) - 3]

    input_ids = [tokenizer_for_bert.cls_token_id] + question_tokens + [tokenizer_for_bert.sep_token_id] + chunk_tokens + [tokenizer_for_bert.sep_token_id]
    attention_mask = [1] * len(input_ids)

    if len(input_ids) > max_len:
        input_ids = input_ids[:max_len]
        attention_mask = attention_mask[:max_len]

    inputs = {
        'input_ids': torch.tensor([input_ids]),
        'attention_mask': torch.tensor([attention_mask])
    }

    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_scores = start_scores.detach().numpy().flatten()
    end_scores = end_scores.detach().numpy().flatten()

    answer_start_index = np.argmax(start_scores)
    answer_end_index = np.argmax(end_scores)

    tokens = tokenizer_for_bert.convert_ids_to_tokens(input_ids)

    answer = tokens[answer_start_index]
    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i].startswith('##'):
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    if (start_scores[answer_start_index] < 0) or (answer_start_index == 0) or (answer_end_index < answer_start_index) or (answer == '[SEP]'):
        answer = "Sorry!, I was unable to discover an answer in the passage."

    return answer, start_scores[answer_start_index] + end_scores[answer_end_index]

def bert_question_answer(question, passage, split_points, max_len=512):

    simplified_question = simplify_question(question)

    parts = split_passage(passage, split_points)

    answers = []
    scores = []

    for part in parts:
        answer, score = process_chunk(simplified_question, part, max_len)
        answers.append(answer)
        scores.append(score)

    # Determine which part has the best score
    if scores:
        best_answer = answers[np.argmax(scores)]
    else:
        best_answer = "Sorry!, I was unable to discover an answer in the passage."

    return best_answer

# Example
passage = "Description: Pancreatic cancer arises when cells in the pancreas.A number of types of pancreatic cancer are known. Several other types of cancer, which collectively represent the majority of the non-adenocarcinomas, can also arise from these cells. Causes: Risk factors for pancreatic cancer include tobacco smoking, obesity, diabetes, and certain rare genetic conditions. Symptoms: Pain in the upper abdomen or back, often spreading from around the stomach to the back, Jaundice, weight loss, Trousseau's syndrome – in which blood clots form spontaneously in the portal blood vessels , weakness and tiring easily, dry mouth, sleep problems, and a palpable abdominal mass. Diagnosis: Histopathology-The most common form of pancreatic cancer (adenocarcinoma) is typically characterized by moderately to poorly differentiated glandular structures on microscopic examination. Staging: Exocrine cancers: Pancreatic cancer is usually staged following a CT scan. Genetic mutations: Pancreatic ductal adenocarcinoma (PDAC):p53, SMAD4, KRAS, p16. Pancreatic acinar cell carcinoma (ACC): p53, SMAD4, ARID1A, GNAS. Solid pseudopapillary tumor:Point mutation in exon 3 of β-catenin gene. Adenosquamous carcinoma: KRAS, p53. Pancreatic neuroendocrine tumor:MEN1, DAXX/ATRX. p53:a tumor suppressor protein encoded by the TP53 gene that responds to DNA damage by regulating cell-cycle arrest, apoptosis, and senescence. SMAD4: It is also known as deleted in pancreatic carcinoma 4 or DPC4, is located on chromosome 18q21 and inactivated in roughly 55% of pancreatic cancers. ARID1A:It a subunit of the SWI/SNF complex, which serves as a tumor suppressor gene commonly absent in subsets of pancreatic cancer and IPMN patients, and low expression of Arid1a is associated with poor overall survival of pancreatic cancer. GNAS: The day-to-day function of the GNAS gene is to maintain body glucose levels and assist in normal development, but when mutated, it becomes overactive and causes cancer. KRAS:Kirsten RAt Sarcoma virus.KRAS is involved in the regulation of cell division as a result of its ability to relay external signals to the cell nucleus MEN1:Multiple endocrine neoplasia type 1 (MEN1) is a rare endocrine tumor syndrome with high penetrance. BRCA: The mutations BRCA1 and BRCA2 increase a person’s lifetime risk of developing pancreatic cancer.Their normal function is to repair damage to DNA, but when BRCA1 or BRCA2 is mutated and doesn’t work correctly, the accumulation of unrepaired DNA damage can ultimately lead to unregulated cell growth, or cancer. Treatments: For cancers involving the head of the pancreas, the Whipple procedure is the most commonly attempted curative surgical treatment. If the cancer is invading or compressing the duodenum or colon, bypass surgery might overcome the obstruction and improve quality of life. Chemotherapy: After surgery, adjuvant chemotherapy with gemcitabine or 5-FU can be offered if the person is sufficiently fit, after a recovery period of one to two months.Radiotherapy: Radiotherapy may form part of treatment to attempt to shrink a tumor to a resectable state, but its use on unresectable tumors remains controversial as there are conflicting results from clinical trials. "

question = "This patient likely has pancreatic cancer. Can you provide information on the role of BRCA mutations in pancreatic cancer and potential implications for treatment?"
split_points = ["Symptoms:", "Genetic mutations:"]

print(bert_question_answer(question, passage, split_points))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Token indices sequence length is longer than the specified maximum sequence length for this model (569 > 512). Running this sequence through the model will result in indexing errors


increase a person ’ s lifetime risk of developing pancreatic cancer


In [ ]:
contexts={"esophageal":"Symptoms: The first and the most common symptom is usually difficulty in swallowing, Pain when swallowing , Weight loss ,Pain behind the breastbone or in the region around the stomach often feels like heartburn,  an unusually husky, raspy, or hoarse-sounding cough, a result of the tumor affecting the recurrent laryngeal nerve, vomiting of blood and Hypercalcemia (excess calcium in the blood). Diagnosis: The disease is diagnosed by biopsy done by an endoscope. Clinical evaluation: diagnosis is best made with an examination using an endoscope. Computed tomography (CT) of the chest, abdomen and pelvis can evaluate whether the cancer has spread to adjacent tissues or distant organs. Positron emission tomography is also used to estimate the extent of the disease and is regarded as more precise than CT alone. Staging: Staging is based on the TNM staging system, which classifies the amount of tumor invasion (T), involvement of lymph nodes (N), and distant metastasis (M). Genetic mutations: TP53 (p53): TP53 is a tumor suppressor gene that plays a crucial role in preventing abnormal cell growth. CDKN2A (p16INK4a):CDKN2A encodes the p16INK4a protein, which regulates the cell cycle by inhibiting cyclin-dependent kinases. NOTCH1: NOTCH1 is involved in cell differentiation and proliferation. Mutations in NOTCH1 have been found in ESCC(esophageal squamous cell carcinoma). SMAD4 (DPC4): SMAD4 is part of the TGF-β signaling pathway, which controls cell growth and differentiation. Mutations in SMAD4 are linked to esophageal adenocarcinoma. PIK3CA: PIK3CA encodes a protein involved in cell growth and survival. Mutations in PIK3CA are seen in both ESCC and adenocarcinoma.HER2: Some esophagus cancers have too much of the HER2 protein on the surface of their cells, which can help cancer cells grow.PD-L1 (Programmed Death-Ligand 1) is a protein expressed on the surface of some cancer cells, including esophageal cancer.In esophageal cancer, the expression of PD-L1 is a critical target for immunotherapy. Drugs known as immune checkpoint inhibitors,block the interaction between PD-L1 and PD-1. By inhibiting this pathway, these drugs help reactivate the immune system, allowing it to recognize and attack the cancer cells.EGFR (epidermal growth factor receptor) mutations in esophageal cancer are less common but can drive tumor growth by activating signaling pathways. These mutations are associated with a more aggressive disease course and Drugs like cetuximab and panitumumab target EGFR to inhibit tumor growth. These are used primarily in cancers where EGFR mutations are present or overexpressed. However, their efficacy in esophageal cancer can vary, and treatment is often combined with chemotherapy or other therapies for improved results.  Treatments: If the cancer has been diagnosed while still in an early stage, surgical treatment with a curative intention may be possible. Curative surgery of early-stage lesions may entail removal of all or part of the esophagus (esophagectomy). Chemotherapy and radiotherapy: Chemotherapy depends on the tumor type, but tends to be cisplatin-based (or carboplatin or oxaliplatin) every three weeks with fluorouracil (5-FU) either continuously or every three weeks. Radiotherapy is given before, during, or after chemotherapy or surgery, and sometimes on its own to control symptoms. In patients with localized disease but contraindications to surgery, radical radiotherapy may be used with curative intent. Laser therapy is the use of high-intensity light to destroy tumor cells while affecting only the treated area. This is typically done if the cancer cannot be removed by surgery. Photodynamic therapy, a type of laser therapy, involves the use of drugs that are absorbed by cancer cells; when exposed to a special light, the drugs become active and destroy the cancer cells.",
          "pancreatic":"Symptoms: Pain in the upper abdomen or back, often spreading from around the stomach to the back, Jaundice, weight loss, weakness and tiring easily, dry mouth, sleep problems, and a palpable abdominal mass. Diagnosis: Histopathology-The most common form of pancreatic cancer (adenocarcinoma) is typically characterized by moderately to poorly differentiated glandular structures on microscopic examination. Staging: Exocrine cancers: Pancreatic cancer is usually staged following a CT scan. Genetic mutations: Pancreatic ductal adenocarcinoma (PDAC):p53, SMAD4, KRAS, p16. Pancreatic acinar cell carcinoma (ACC): p53, SMAD4, ARID1A, GNAS. Solid pseudopapillary tumor: Point mutation in exon 3 of β-catenin gene. Adenosquamous carcinoma: KRAS, p53. Pancreatic neuroendocrine tumor:MEN1, DAXX/ATRX. ATRX: cause genomic instability, activate the ALT pathway, worsen prognosis, and guide the development of targeted therapies and biomarkers for improved treatment strategies.  MEN1:Multiple endocrine neoplasia type 1 (MEN1) is a rare endocrine tumor syndrome with high penetrance.  CDKN2A mutations in pancreatic cancer disable cell cycle control, promoting tumor growth and Treatment may involve CDK4/6 inhibitors to restore regulation. BRCA: The mutations BRCA1 and BRCA2 increase a person’s lifetime risk of developing pancreatic cancer.Their normal function is to repair damage to DNA, but when BRCA1 or BRCA2 is mutated and doesn’t work correctly, the accumulation of unrepaired DNA damage can ultimately lead to unregulated cell growth, or cancer. PTEN mutations in pancreatic cancer lead to unchecked cell growth via the PI3K/AKT pathway, with treatment options including PI3K inhibitors. PBRM1: disrupt chromatin remodeling, contributing to tumor growth and Treatment may focus on targeting epigenetic alterations or using combination therapies with immune checkpoint inhibitors. ARID2: lead to chromatin remodeling defects, driving tumor progression and Treatment implications may involve targeting epigenetic changes or using immunotherapies that exploit the tumor's altered gene expression. p53:a tumor suppressor protein encoded by the TP53 gene that responds to DNA damage by regulating cell-cycle arrest, apoptosis, and senescence. ATM:People with an ATM mutation have an increased risk for pancreatic cancer. SMAD4: It is also known as deleted in pancreatic carcinoma 4 or DPC4, is located on chromosome 18q21 and inactivated in roughly 55% of pancreatic cancers and plays a crucial role in maintaining normal cell behavior, and its loss in pancreatic cancer is associated with more aggressive disease and poorer outcomes. ARID1A:It a subunit of the SWI/SNF complex, which serves as a tumor suppressor gene commonly absent in subsets of pancreatic cancer and IPMN patients, and low expression of Arid1a is associated with poor overall survival of pancreatic cancer. KRAS:Kirsten RAt Sarcoma virus.KRAS is involved in the regulation of cell division as a result of its ability to relay external signals to the cell nucleus. BAP1: lead to impaired DNA repair and increased tumor aggressiveness. Treatment implications may include using PARP inhibitors, which target the DNA repair deficiencies caused by BAP1 loss.MEN2 mutations are primarily linked to medullary thyroid cancer but can also play a role in pancreatic cancer by promoting abnormal cell growth. Treatment may involve targeted therapies like RET inhibitors. NTRK gene fusions involve the fusion of one of the NTRK genes (NTRK1, NTRK2, or NTRK3) with another gene, leading to the production of a chimeric TRK protein. This fusion protein is constitutively active, leading to uncontrolled signaling through pathways involved in cell growth and survival, contributing to cancer development.The discovery of NTRK gene fusions in cancers, including pancreatic cancer, has led to the development of targeted therapies known as TRK inhibitors. These inhibitors are designed to specifically target the abnormal TRK fusion proteins, blocking their activity and thereby inhibiting cancer cell growth and survival. ATM (Ataxia Telangiectasia Mutated) mutations in pancreatic cancer are associated with defects in DNA repair mechanisms. This can lead to increased genomic instability and contribute to cancer progression and ATM mutations may make tumors more susceptible to DNA-damaging agents. This includes platinum-based chemotherapies, which exploit the DNA repair deficiencies.PALB2 (Partner and Localizer of BRCA2) mutations in pancreatic cancer are associated with impaired DNA repair, similar to BRCA2 mutations. This can lead to increased susceptibility to pancreatic cancer and other malignancies.Tumors with PALB2 mutations may respond better to platinum-based chemotherapies due to their compromised DNA repair pathways. STK11, also known as LKB1, is a tumor suppressor gene that plays a crucial role in regulating cellular energy metabolism, growth, and response to stress. In pancreatic cancer, STK11 mutations are associated with poor prognosis and increased aggressiveness. These mutations contribute to cancer progression by disrupting cellular homeostasis and promoting resistance to therapies and STK11 mutations have been linked to low levels of tumor-infiltrating lymphocytes and poor response to immune checkpoint inhibitors like PD-1/PD-L1 inhibitors. However, emerging research is investigating ways to enhance immunotherapy efficacy in STK11-mutant cancers. CDKN2A mutations in pancreatic cancer disable cell cycle control, promoting tumor growth and Treatment may involve CDK4/6 inhibitors to restore regulation. Treatments: For cancers involving the head of the pancreas, the Whipple procedure is the most commonly attempted curative surgical treatment. If the cancer is invading or compressing the duodenum or colon, bypass surgery might overcome the obstruction and improve quality of life. Chemotherapy: After surgery, adjuvant chemotherapy with gemcitabine or 5-FU can be offered if the person is sufficiently fit, after a recovery period of one to two months.Radiotherapy: Radiotherapy may form part of treatment to attempt to shrink a tumor to a resectable state, but its use on unresectable tumors remains controversial as there are conflicting results from clinical trials.",
          "gall bladder":"Symptoms: Steady pain in the upper right abdomen, Indigestion (dyspepsia),Bilious vomit,Weakness, Loss of appetite, Weight loss, Jaundice and vomiting due to obstruction, Early symptoms mimic gallbladder inflammation due to gallstones. Diagnosis: Transabdominal ultrasound, CT scan, endoscopic ultrasound, MRI, and MR cholangio-pancreatography (MRCP) can be used for diagnosis. A biopsy is the only certain way to tell whether or not the tumorous growth is malignant. Genetic mutations: Gallbladder carcinomas most frequently harbor alterations in TP53, CDKN2A, SMAD4, ARID1A, and CDKN2B. TP53 (p53): TP53 is a tumor suppressor gene that plays a crucial role in preventing abnormal cell growth. CDKN2A: It encodes CDKI p16 whose function is to inhibit cyclin D1-dependent kinase CDK4 from interacting with cyclin D1 and phosphorylating Rb. Rb hyperphosphorylation would lead to cell cycle progression from G1 to S. SMAD4: It is also known as deleted in pancreatic carcinoma 4 or DPC4, is located on chromosome 18q21 and inactivated in roughly 55% of pancreatic cancers. ARID1A: It a subunit of the SWI/SNF complex, which serves as a tumor suppressor gene commonly absent in subsets of pancreatic cancer and IPMN patients, and low expression of Arid1a is associated with poor overall survival of pancreatic cancer. Treatments: Surgery for gallbladder cancer is called radical cholecystectomy or extended cholecystectomy. It entails the removal of gallbladder along with adequate removal of its liver bed to the healthy tissue. The lymph nodes in the vicinity are also removed. Sometimes removal of a large part of the liver called hepatectomy is required to completely remove the tumor. The bile duct if involved also needs to be removed. If surgery is not possible, endoscopic stenting or percutaneous transhepatic biliary drainage (PTBD) of the biliary tree can reduce jaundice and a stent in the stomach may relieve vomiting. Chemotherapy and radiation may also be used with surgery.",
          "stomach":"Symptoms: Early symptoms may include heartburn, upper abdominal pain, nausea, and loss of appetite. Later signs and symptoms may include weight loss, yellowing of the skin and whites of the eyes, vomiting, difficulty swallowing, and blood in the stool, among others. Diagnosis: It is usually by biopsy done during endoscopy. This is followed by medical imaging to determine if the disease has spread to other parts of the body.Gastroscopic exam is the diagnostic method of choice. This involves insertion of a fiber optic camera into the stomach to visualize it. Upper GI series (may be called barium roentgenogram) Computed tomography or CT scanning of the abdomen may reveal gastric  cancer. Genetic mutations: A genetic risk factor for gastric cancer is a genetic defect of the CDH1 gene known as hereditary diffuse gastric cancer (HDGC). The CDH1 gene, which codes for E-cadherin, lies on the 16th chromosome. A very small percentage of diffuse-type gastric cancers arise from an inherited abnormal CDH1 gene. Treatments: Treatment for stomach cancer may include surgery, chemotherapy, or radiation therapy. New treatment approaches such as immunotherapy or gene therapy and improved ways of using current methods are being studied in clinical trials.Endoscopic mucosal resection (EMR) is a treatment for early gastric cancer. Some drugs used in stomach cancer treatment have included:fluorouracil or its analog capecitabine, BCNU (carmustine), methyl-CCNU (semustine) and doxorubicin (Adriamycin), as well as mitomycin C, and more recently cisplatin and taxotere, often using drugs in various combinations. Surgery is the most frequent treatment option for stomach cancers but it is associated with significant risks.",
          "liver":"Symptoms: Some general symptoms are lump or pain in the right side below the rib cage, swelling of the abdomen, yellowish skin, easy bruising, weight loss, weakness. Cholangiocarcinoma is associated with sweating, jaundice, abdominal pain, weight loss, and liver enlargement. Hepatocellular carcinoma is associated with abdominal mass, abdominal pain, vomiting, anemia, back pain, jaundice, itching, weight loss and fever. Symptoms for Intrahepatic cholangiocarcinoma include abdominal pain, night sweats, weight loss, and fatigue. Diagnosis: For HCC these include medical ultrasound, computed tomography (CT) and magnetic resonance imaging (MRI) and liver biopsy. If it is suspected to be malignant, endoscopic retrograde cholangiopancreatography (ERCP), ultrasound, CT, MRI and magnetic resonance cholangiopancreatography (MRCP) are used. High levels of alpha-fetoprotein (AFP) in the blood can be found in many cases of HCC and intrahepatic cholangiocarcinoma. AFP is most useful for monitoring if liver cancers come back after treatment rather than for initial diagnosis.Cholangiocarcinoma can be detected with these commonly used tumor markers: carbohydrate antigen 19-9 (CA 19–9), carcinoembryonic antigen (CEA) and cancer antigen 125 (CA125). Genetic mutations: Many genetic and epigenetic changes are formed in liver cells during HCV and HBV infection, which is a major factor in the production of liver tumors. Genetic conditions: untreated hereditary hemochromatosis, alpha-1-antitrypsin deficiency, glycogen storage diseases, porphyria cutanea tarda, Wilson's disease, tyrosinemia have all been associated with development of HCC. Hemochromatosis: An inherited condition in which too much iron builds up in the body. This can cause liver disease, heart problems, diabetes, and organ failure. Treatment: Treatments include surgery, medications, and ablation methods. There are many chemotherapeutic drugs approved for liver cancer including: atezolizumab, nivolumab, pembrolizumab, and regorafenib. Increasingly, immunotherapy agents (also called targeted cancer therapies or precision medicine) are being used to treat hepatobiliary cancers. Liver transplantation can also be considered in cases of HCC where this form of treatment can be tolerated and the tumor fits specific criteria (such as the Milan criteria). Photodynamic therapy: The compounds are activated in the tumor region by laser light, which causes the release of toxic reactive oxygen species, killing tumor cells.Radio frequency ablation, transarterial chemoembolization and internal radiotherapy (brachytherapy) all show promise in the treatment of cholangiocarcinoma and can sometimes improve bile flow, which can decrease the symptoms a patient experiences. Radiotherapy may be used in the adjuvant setting or for palliative treatment of cholangiocarcinoma. Removing the tumor by either surgical resection or liver transplant can be used in the treatment of hepatoblastoma. In some cases surgery can offer a cure. Chemotherapy, including cisplatin, vincristine, cyclophosphamide, and doxorubicin are used for the systemic treatment of hepatoblastoma. Out of these drugs, cisplatin seems to be the most effective.",
          "anal":"Symptoms: Symptoms include pain or pressure in the anus or rectum, a change in bowel habits, a lump near the anus, rectal bleeding, itching or discharge. Bleeding may be severe, pain, itchiness, or discharge from the anus.Diagnosis: It is suspected based on physical examination and confirmed by tissue biopsy. Genetic mutations: Anal carcinomas most frequently harbor alterations in PIK3CA, KMT2D, SOX2, and BCL6. PIK3CA:PIK3CA codes for the catalytic subunit of phosphoinositide 3-kinase alpha (PI3Kα), which ultimately activates the AKT and mammalian target of rapamycin (mTOR) pathway. KMT2D:It encodes a histone lysine methyltransferase whose mutations are associated with a variety of cancers, blood malignancies in particular, and are considered markers of poor prognosis. SOX2:It is an undifferentiated cell marker. Recent studies have shown that Sox2 may also be involved in the maintenance of cancer stem cells (CSCs) in skin and bladder cancers. BCL6:BCL6 is a transcriptional repressor required in mature B cells during the germinal center (GC) reaction. The critical functions exerted by BCL6 during normal B-cell development can be hijacked by the malignant transformation process, thus indicating its critical involvement in colorectal cancer. Treatment: Standard treatment may include radiation therapy, chemotherapy, and surgery. Current gold-standard therapy is the combination of chemotherapy and radiation treatment to reduce the necessity of debilitating surgery. Biopsies to document disease regression after chemotherapy and radiation were commonly advised, but are not as frequent any longer. Current chemotherapy consists of continuous infusion 5-FU over four days with bolus mitomycin given concurrently with radiation. 5-FU and cisplatin are recommended for metastatic anal cancer.",
          "colorectal":"Symptoms: Symptoms may include blood in the stool, a change in bowel movements, weight loss, abdominal pain and fatigue. Also, worsening constipation, blood in the stool, decrease in stool caliber (thickness), loss of appetite, loss of weight, and nausea or vomiting. Chronic inflammation is a recognized risk factor for colorectal cancer. Diagnosis: Colorectal cancer diagnosis is performed by sampling of areas of the colon suspicious for possible tumor development, typically during colonoscopy. Presence of metastases is determined by a CT scan of the chest, abdomen and pelvis. MRI is particularly useful to determine the local stage of the tumor and to plan the optimal surgical approach. Patients selected for non-surgical treatment of rectal cancer should have periodic MRI scans, receive physical examinations, and undergo endoscopy procedures to detect any tumor regrowth which can occur in a minority of these patients. Genetic mutations: Those with a family history in two or more first-degree relatives (such as a parent or sibling) have a two to threefold greater risk of disease. The most common of these is hereditary nonpolyposis colorectal cancer (HNPCC, or Lynch syndrome) which is present in about 3% of people with colorectal cancer. Mutations in the pair of genes (POLE and POLD1) have been associated with familial colon cancer. A gene that appears to contribute to the potential for metastatic disease, metastasis associated in colon cancer 1 (MACC1), has been isolated. It is a transcriptional factor that influences the expression of hepatocyte growth factor. This gene is associated with the proliferation, invasion, and scattering of colon cancer cells in cell culture, and tumor growth and metastasis in mice. MACC1 may be a potential target for cancer intervention, but this possibility needs to be confirmed with clinical studies.Approximately 30-50% of colorectal tumors are known to have a mutated (abnormal) KRAS gene, indicating that up to 50% of patients with colorectal cancer (CRC) might respond to anti-epidermal growth factor receptor (EGFR) antibody therapy. However, 40-60% of patients with wild-type KRAS tumors do not respond to such therapy. In these patients, data suggest that the mutated BRAF gene, which is present in 5-10% of tumors, can affect response to these agents.Chronic inflammation in colorectal cancer refers to a prolonged immune response in the colon or rectum, leading to tissue damage and promoting cancer development. Persistent inflammation can cause genetic mutations, alter cell behavior, and create a tumor-friendly environment, increasing the risk of colorectal cancer over time.Microsatellite instability (MSI) is a condition of genetic hypermutability that results from impaired DNA mismatch repair (MMR). It occurs when the MMR system fails to correct errors that occur during DNA replication, leading to the accumulation of mutations, particularly in microsatellite regions (short, repetitive DNA sequences).MSI-H tumors are less responsive to fluoropyrimidine-based chemotherapy (e.g., 5-fluorouracil) when used alone, particularly in stage II colon cancer. This has led to a re-evaluation of the use of adjuvant chemotherapy in patients with stage II MSI-H colorectal cancer.The BRAF gene mutation is found in about 10-15% of colorectal cancers. This mutation activates signaling pathways that drive cancer cell growth and contribute to aggressive tumor behavior.These tumors often do not respond well to conventional chemotherapy. However, targeted therapies such as BRAF inhibitors (e.g., vemurafenib) and combination treatments with MEK inhibitors and EGFR inhibitors can offer improved outcomes. PIK3CA mutations activate the PI3K/Akt signaling pathway, which promotes tumor growth and survival. These mutations are found in a subset of colorectal cancers and are associated with poor prognosis and resistance to some therapies.Targeting the PI3K pathway with specific inhibitors like idelalisib or copanlisib can help manage tumors with PIK3CA mutations.PTEN (phosphatase and tensin homolog) mutations lead to the loss of PTEN function, which results in increased activation of the PI3K/Akt signaling pathway. This promotes tumor growth and resistance to various therapies. PTEN mutations are linked to poor prognosis and can contribute to the development of colorectal cancer and Since PTEN mutations lead to uncontrolled activation of this pathway, inhibitors targeting PI3K (e.g., idelalisib), mTOR (e.g., everolimus), or Akt can be used to counteract the effects.Treatment: When colorectal cancer is caught early, surgery can be curative. However, when it is detected at later stages (for which metastases are present), this is less likely and treatment is often directed at palliation, to relieve symptoms caused by the tumor.At an early stage, colorectal cancer may be removed during a colonoscopy. The procedure of choice is a partial colectomy (or proctocolectomy for rectal lesions) where the affected part of the colon or rectum is removed along with parts of its mesocolon and blood supply to facilitate removal of draining lymph nodes. The colon may then be reconnected or a person may have a colostomy. Chemotherapy may be used before surgery to shrink the cancer before attempting to remove it. In Stage I colon cancer, no chemotherapy is offered, and surgery is the definitive treatment. The role of chemotherapy in Stage II colon cancer is debatable, and is usually not offered unless risk factors such as T4 tumor, undifferentiated tumor, vascular and perineural invasion or inadequate lymph node sampling is identified. For Stage III and Stage IV colon cancer, chemotherapy is an integral part of treatment. For patients with stage III colorectal cancer, where the cancer has spread to nearby lymph nodes but not to distant sites, adjuvant chemotherapy is a standard part of treatment after surgery. The goal of adjuvant chemotherapy is to eliminate any remaining cancer cells that may not have been removed during surgery, thereby reducing the risk of cancer recurrence.",
          "GIST":"Symptoms:GISTs may present with trouble swallowing, gastrointestinal bleeding, or metastases (mainly in the liver) and vague abdominal pain or discomfort. Diagnosis:CT scanning is often undertaken. The definitive diagnosis is made with a biopsy, which can be obtained endoscopically, percutaneously with CT or ultrasound guidance or at the time of surgery. Genetic mutations:The Carney-Stratakis syndrome is a dyad of hereditary GIST and paraganglioma, that is caused by germline mutations in the mitochondrial tumor suppressor gene pathway involving the succinate dehydrogenase (SDH) subunits SDHD, SDHC and SDHB. SDHC mutations in GIST impair the succinate dehydrogenase (SDH) complex, leading to tumorigenesis and Treatment options are limited, but potential approaches include targeting metabolic pathways or using SDH-related inhibitors. NF1 is a tumor suppressor gene that negatively regulates the RAS signaling pathway. Loss of NF1 function leads to increased RAS activation, which can drive tumor growth and resistance to certain therapies and This tyrosine kinase inhibitor, which targets KIT and PDGFRA mutations commonly found in GISTs, might be less effective in NF1-mutant GISTs due to different underlying molecular mechanisms. TET2: affect DNA methylation and tumor progression and Targeted therapy may involve agents that reverse epigenetic changes or specific inhibitors. MEN1: disrupt tumor suppressor functions and Targeted therapy options may include agents that address these genetic disruptions or related signaling pathways. c-KIT is a gene that encodes for a transmembrane receptor for a growth factor termed stem cell factor (scf). The abnormal c-KIT pathway most commonly arises from mutation of the gene itself. Mutations generally occur in the DNA encoding the intracellular part (exon 11), which acts as a tyrosine kinase to activate other enzymes. Mutations make c-KITfunction independent of activation by scf, leading to a high cell division rate and possibly genomic instability. Mutations in the exons 11, 9 and rarely 13 and 17 of the c-KIT gene are known to occur in GIST. The tyrosine kinase function of c-KIT is important in the medical therapy for GISTs: KIT-D816V point mutations in c-KIT exon 17 are responsible for resistance to targeted therapy drugs like imatinib mesylate, a tyrosine kinase inhibitor.KIT-p.D419del (exon 8) — A subset of gastrointestinal stromal tumors previously regarded as wild-type tumors carries somatic activating mutations in KIT exon 8 .Most GIST cells with wildtype (i.e. not mutated) c-KIT instead have a mutation in another gene, PDGFR-α (platelet-derived growth factor receptor alpha), which is a related tyrosine kinase. Mutations in c-KIT and PDGFrA are mutually exclusive.Mutations in the CDKN2A gene in GIST can lead to loss of cell cycle regulation, promoting tumor growth Targeted therapy options often focus on KIT and PDGFRA mutations but may be less effective in cases involving CDKN2A mutations, necessitating alternative strategies like CDK inhibitors. CDK4: lead to unchecked cell cycle progression and tumor growth. Targeted therapy options may include CDK4/6 inhibitors, which specifically block this pathway to control tumor proliferation. SDHAF2: disrupt the function of the succinate dehydrogenase (SDH) complex, leading to tumorigenesis and Targeted therapies are limited, but potential options include metabolic pathway inhibitors and epigenetic therapies. CHEK2 mutations in GIST can lead to impaired DNA damage response, contributing to tumor progression and Targeted therapy options may include agents that exploit this vulnerability, such as PARP inhibitors. TSC1: disrupt the mTOR signaling pathway, leading to increased cell growth and tumorigenesisand targeted therapy options include mTOR inhibitors like everolimus to counteract this pathway activation. The NF2 gene which acts as a tumor suppressor by regulating cell growth and maintaining cell shape.GISTs are often treated with TKIs like imatinib, sunitinib, or regorafenib, especially when they involve KIT or PDGFRA mutations. However, the effectiveness of TKIs in NF2-mutated GISTs is less clear due to the rarity of these mutations.SDHB mutations in GIST disrupt the SDH complex, driving tumor growth and Targeted therapies may include metabolic inhibitors or SDH-targeted treatments. Treatments:surgery is the primary treatment of choice.Post-surgical adjuvant treatment may be recommended.Laparoscopic surgery, a minimally invasive abdominal surgery using telescopes and specialized instruments, has been shown to be effective for removal of these tumors without needing large incisions"
          }


In [ ]:
question="This patient likely has pancreatic cancer. Can you provide information on the role of MEN1 mutations in pancreatic cancer pathogenesis and treatment implications?"
keys=list(contexts.keys())
context=""
for i in keys:
  if i in question:
    context=contexts[i]
#print(context)

passage = context

split_points = ["Symptoms:", "Genetic mutations:"]

print(bert_question_answer(question, passage, split_points))

Sorry!, I was unable to discover an answer in the passage.
